In [1]:
#importing all the required lib
import pandas as pd
import numpy as np 
import os
from keras.preprocessing import sequence
import math
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import SpatialDropout1D, LSTM, BatchNormalization,concatenate,Flatten,Embedding,Dense,Dropout,MaxPooling2D,Reshape,LSTM
from keras.models import Sequential
from keras import Model,Input
from keras.layers.convolutional import Conv2D,Conv1D
import keras.backend as k
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import keras
from numpy import zeros
from sklearn.utils import compute_class_weight
from keras.initializers import he_normal,glorot_normal
from keras.regularizers import l1,l2
from tensorflow.python.keras.callbacks import TensorBoard
from IPython.display import SVG, display
import pickle 
import warnings
from tqdm.auto import tqdm
warnings.filterwarnings("ignore")

In [2]:
#reading the data
data=pd.read_csv('/kaggle/input/donor-data/preprocessed_donor_data.csv')
data.head(3)

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,725.05
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,213.03
2,ca,mrs,grades_prek_2,10,1,literacy_language,literacy,having class 24 students comes diverse learner...,329.00


In [3]:
X = data.drop(['project_is_approved'], axis=1)
Y = data['project_is_approved'].values

In [4]:
#split data into train , test, cv 
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, stratify=Y)

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(81936, 8) (81936,)
(27312, 8) (27312,)


In [5]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words= 100000 )
tokenizer.fit_on_texts(X_train["essay"])

X_train['essay_tok']  = tokenizer.texts_to_sequences(X_train['essay'].values)
X_test['essay_tok']   = tokenizer.texts_to_sequences(X_test['essay'].values)

In [6]:
from keras.preprocessing import sequence
max_review_length = 300
X_train_pad = sequence.pad_sequences(X_train['essay_tok'].values, maxlen=max_review_length , padding='post')
X_test_pad  = sequence.pad_sequences(X_test['essay_tok'].values, maxlen=max_review_length , padding='post')

In [7]:
dbfile = open('../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl', 'rb')      
db = pickle.load(dbfile)

In [8]:
glove_words =  set(db.keys())

In [9]:
from numpy import zeros
max_vocabulary = len(tokenizer.word_index)
# create a weight matrix for words in training docs
embedding_matrix = zeros((max_vocabulary+1, 300))
for word, i in tqdm(tokenizer.word_index.items()):
    if word in glove_words:
        embedding_vector = db[word]
        embedding_matrix[i] = embedding_vector

# School State

In [10]:
from sklearn.preprocessing import LabelEncoder
labele = LabelEncoder()
labele.fit(X['school_state'].values)


X_train_school_state = labele.transform(X_train['school_state'].values)

X_test_school_state = labele.transform(X_test['school_state'].values)


no_of_unique_states = X_train['school_state'].nunique()
print("Number of unique school state= ",no_of_unique_states)

embedding_size_school_state = min(np.ceil((no_of_unique_states)/2), 50 )
embedding_size_school_state = int(embedding_size_school_state)
print('Embedding size = ',embedding_size_school_state)

Number of unique school state=  51
Embedding size =  26


# Teacher Prefix

In [11]:
labele = LabelEncoder()
labele.fit(X['teacher_prefix'].values)

X_train_teacher_prefix = labele.transform(X_train['teacher_prefix'].values)

X_test_teacher_prefix = labele.transform(X_test['teacher_prefix'].values)


no_of_unique_teacher_prefix = X_train['teacher_prefix'].nunique()
print("Number of unique teacher_prefix= ",no_of_unique_teacher_prefix)

embedding_size_teacher_prefix = min(np.ceil((no_of_unique_teacher_prefix)/2), 50 )
embedding_size_teacher_prefix = int(embedding_size_teacher_prefix)
print('Embedding size = ',embedding_size_teacher_prefix)

Number of unique teacher_prefix=  5
Embedding size =  3


# Clean Categories

In [12]:
labele = LabelEncoder()
labele.fit(X['clean_categories'].values)


X_train_clean_categories = labele.transform(X_train['clean_categories'].values)

X_test_clean_categories = labele.transform(X_test['clean_categories'].values)

no_of_unique_clean_categories = X_train['clean_categories'].nunique()
print("Number of unique clean_categories= ",no_of_unique_clean_categories)

embedding_size_clean_categories = min(np.ceil((no_of_unique_clean_categories)/2), 50 )
embedding_size_clean_categories = int(embedding_size_clean_categories)
print('Embedding size = ',embedding_size_clean_categories)


Number of unique clean_categories=  51
Embedding size =  26


# Clean Subcategories

In [13]:
labele = LabelEncoder()
labele.fit(X['clean_subcategories'].values)


X_train_clean_subcategories = labele.transform(X_train['clean_subcategories'].values)

X_test_clean_subcategories = labele.transform(X_test['clean_subcategories'].values)

no_of_unique_clean_subcategories = X_train['clean_subcategories'].nunique()
print("Number of unique clean_subcategories= ",no_of_unique_clean_subcategories)


embedding_size_clean_subcategories = min(np.ceil((no_of_unique_clean_subcategories)/2), 50 )
embedding_size_clean_subcategories = int(embedding_size_clean_subcategories)
print('Embedding size = ',embedding_size_clean_subcategories)

Number of unique clean_subcategories=  391
Embedding size =  50


# Project Grade Category

In [14]:
labele = LabelEncoder()
labele.fit(X['project_grade_category'].values)


X_train_project_grade_category = labele.transform(X_train['project_grade_category'].values)

X_test_project_grade_category = labele.transform(X_test['project_grade_category'].values)

no_of_unique_project_grade_category = X_train['project_grade_category'].nunique()
print("Number of unique project_grade_category= ",no_of_unique_project_grade_category)

embedding_size_project_grade_category = min(np.ceil((no_of_unique_project_grade_category)/2), 50 )
embedding_size_project_grade_category = int(embedding_size_project_grade_category)
print('Embedding size = ',embedding_size_project_grade_category)

Number of unique project_grade_category=  4
Embedding size =  2


In [15]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))


X_train_num_projects=scaler.transform(X_train['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))

X_test_num_projects=scaler.transform(X_test['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))

In [16]:
scaler.fit(X_train['price'].values.reshape(-1,1))

X_train_price = scaler.transform(X_train['price'].values.reshape(-1,1))

X_test_price = scaler.transform(X_test['price'].values.reshape(-1,1))

In [17]:
X_train_numeric_features = np.concatenate((X_train_num_projects , X_train_price) , axis = 1)

X_test_numeric_features= np.concatenate((X_test_num_projects , X_test_price) , axis = 1)

print(X_train_numeric_features.shape ,X_test_numeric_features.shape)

(81936, 2) (27312, 2)


In [18]:
def auc1(y_true, y_pred):
    if len(np.unique(y_true[:,1])) == 1:
        return 0.5
    else:
        return roc_auc_score(y_true, y_pred)

def auroc(y_true, y_pred):
    return tf.py_function(auc1, (y_true, y_pred), tf.double)

In [19]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(Y_train, 2) 
y_test = np_utils.to_categorical(Y_test, 2)

In [20]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras import regularizers
from keras.regularizers import l2
from keras.layers import Flatten
from keras.layers import Dense, Input , Dropout
from keras.layers import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard

In [21]:
tf.keras.backend.clear_session()

Input_1 = Input(shape=(300,), name='essay_input')

X = Embedding(output_dim=300, input_dim=max_vocabulary+1, input_length=300 , weights=[embedding_matrix])(Input_1)
lstm_essay = LSTM(100,recurrent_dropout=0.5,kernel_regularizer=regularizers.l2(0.001),return_sequences=True)(X)
flatten_1 = Flatten()(lstm_essay)



Input_2 = Input(shape=(1,), name='school_state')
X_school_state = Embedding(output_dim=embedding_size_school_state , input_dim=no_of_unique_states, input_length=1)(Input_2)
flatten_2 = Flatten()(X_school_state)



Input_3 = Input(shape=(1,), name='teacher_prefix')
X_teacher_prefix = Embedding(output_dim=embedding_size_teacher_prefix , input_dim=no_of_unique_teacher_prefix, input_length=1)(Input_3)
flatten_3 = Flatten()(X_teacher_prefix)



Input_4 = Input(shape=(1,), name='clean_categories')
X_clean_categories = Embedding(output_dim=embedding_size_clean_categories, input_dim=no_of_unique_clean_categories, input_length=1)(Input_4)
flatten_4 = Flatten()(X_clean_categories)



Input_5 = Input(shape=(1,), name='clean_subcategories')
X_clean_subcategories = Embedding(output_dim=embedding_size_clean_subcategories, input_dim=500, input_length=1)(Input_5)
flatten_5 = Flatten()(X_clean_subcategories)




Input_6 = Input(shape=(1,), name='project_grade_category')
X_project_grade_category = Embedding(output_dim=embedding_size_project_grade_category, input_dim=no_of_unique_project_grade_category, input_length=1)(Input_6)
flatten_6 = Flatten()(X_project_grade_category)


Input_7 = Input(shape=(2,) , name="numerical_features")
numeric_dense = Dense(128, activation='relu' , kernel_initializer='he_normal',kernel_regularizer=l2(0.001))(Input_7)


X_concat = concatenate([flatten_1 , flatten_2 , flatten_3 ,flatten_4 , flatten_5 , flatten_6 , numeric_dense])
model = Dense(300, activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=regularizers.l2(0.001))(X_concat)

model = Dropout(0.5)(model)

model = Dense(200,activation="relu",kernel_initializer="glorot_normal" ,kernel_regularizer=regularizers.l2(0.001))(model)

model = BatchNormalization()(model)

model = Dropout(0.5)(model)

model = Dense(80,activation="relu", kernel_initializer="glorot_normal" ,kernel_regularizer=regularizers.l2(0.001))(model)


output = Dense(2, activation='softmax', name='output')(model)

model_1 = Model(inputs=[Input_1,Input_2,Input_3,Input_4,Input_5,Input_6,Input_7],outputs=[output])



print(model_1.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
essay_input (InputLayer)        [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 300, 300)     15126000    essay_input[0][0]                
__________________________________________________________________________________________________
school_state (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
teacher_prefix (InputLayer)     [(None, 1)]          0                                            
_______________________________________________________________________________________

In [22]:

from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau, EarlyStopping
checkpoint1 = ModelCheckpoint("model_1.h5",
                             monitor="val_auroc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)
earlystop1 = EarlyStopping(monitor = 'val_auroc', 
                            mode="max",
                            min_delta = 0, 
                            patience = 2,
                            verbose = 1)

tensorboard1 = TensorBoard(log_dir='Model1_visualization')

callbacks_1 = [checkpoint1,earlystop1,tensorboard1]

In [23]:

x_train = [X_train_pad,X_train_school_state,X_train_teacher_prefix,X_train_clean_categories,X_train_clean_subcategories,X_train_project_grade_category,X_train_numeric_features]

x_test = [X_test_pad,X_test_school_state,X_test_teacher_prefix,X_test_clean_categories,X_test_clean_subcategories,X_test_project_grade_category,X_test_numeric_features]

In [24]:
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[auroc])
history = model_1.fit(x_train, y_train, batch_size=512, epochs=10, verbose=1,callbacks=callbacks_1, validation_data=(x_test, y_test))

Epoch 1/10
161/161 [==============================] - ETA: 0s - loss: 1.4234 - auroc: 0.6361
Epoch 00001: val_auroc improved from -inf to 0.65560, saving model to model_1.h5
161/161 [==============================] - 231s 1s/step - loss: 1.4234 - auroc: 0.6361 - val_loss: 0.9899 - val_auroc: 0.6556
Epoch 2/10
161/161 [==============================] - ETA: 0s - loss: 0.8604 - auroc: 0.7198
Epoch 00002: val_auroc improved from 0.65560 to 0.75232, saving model to model_1.h5
161/161 [==============================] - 226s 1s/step - loss: 0.8604 - auroc: 0.7198 - val_loss: 0.7446 - val_auroc: 0.7523
Epoch 3/10
161/161 [==============================] - ETA: 0s - loss: 0.6749 - auroc: 0.7572
Epoch 00003: val_auroc did not improve from 0.75232
161/161 [==============================] - 223s 1s/step - loss: 0.6749 - auroc: 0.7572 - val_loss: 0.6333 - val_auroc: 0.7439
Epoch 4/10
161/161 [==============================] - ETA: 0s - loss: 0.5812 - auroc: 0.7863
Epoch 00004: val_auroc did not im

In [25]:
y_test_pred = model_1.predict(x_test)
print("Test AUC:",roc_auc_score(y_test,y_test_pred))

Test AUC: 0.7513793251155567
